### ML PYspark

The first thing to do is to create a .env file in the root of the directory. Add to the file the following two varibles 
ACCESS_KEY, ACCESS_SECRET. 
Check for more detailed explanation here: [dotenv]("https://pypi.org/project/python-dotenv/), he explains how the .env should look like. After that, the variables are add to the os.environ and can be access as a simple dict structure

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as spark_sum, when, input_file_name
from functools import reduce
import sys
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StandardScaler
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()

## check
print(f"{os.environ['ACCESS_KEY']}")

## Console Login

The following classes are to handle the spark on the AWS 

In [ ]:
from src.s3handler import Sparker

In [ ]:
## Initialize the class
spark = Sparker(os.environ['ACCESS_KEY'],os.environ['ACCESS_SECRET'])

## local session
spark._create_local_session()

In [ ]:
## Read the parquet and stored it 
df = spark.read_parquet("ubs-datasets",
                    "FRACTAL/data/train/TRAIN-0436_6399-002955400.parquet",
                    read_all=False)

# # Read the list of parquet files
# list_s3 = ["FRACTAL/data/train/TRAIN-1200_6136-008972557.parquet", "FRACTAL/data/train/TRAIN-0436_6399-002955400.parquet"]
# df = spark.read_parquet("ubs-datasets",
#                     list_s3,
#                     read_all=False)

In [ ]:
df.printSchema()

see that the schema here was infered by the spark and it is totally different from when I had to download the file

In [ ]:
df.head()

In [ ]:
print(f"Number of rows: {df.count()}")

In [ ]:
conditions = [col(c).isNull() for c in df.columns]

##combined condition returns True for any \
# row where at least one column is NULL
combined_condition = reduce(lambda a, b: a | b, conditions)

print(f"Number of cols with null values:{df.filter(combined_condition).count()}")

In [ ]:
from pyspark.ml.feature import StandardScaler

In [ ]:
# Split the array column into three separate columns
df = df.withColumn("x", col("xyz")[0]) \
       .withColumn("y", col("xyz")[1]) \
       .withColumn("z", col("xyz")[2])

In [ ]:
scaler = StandardScaler(inputCol="features",
                        outputCol="scaledFeatures",
                        withStd=True,
                        withMean=False)

In [ ]:
## Select cols
feature_cols = ['x','y','z', 'Intensity', 'Red','Green','Blue','Infrared']  

## Create an Vector Assembler
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

## scaler
scaler = StandardScaler(inputCol="features", outputCol="scaled_features")


In [ ]:
output = assembler.transform(df)

In [ ]:
output.select("features").show(truncate=False)